In [ ]:
import numpy as np
from scipy.fftpack import dct
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

def local_noise_var(I):
    I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    I = I.astype(float)

    width, height = I.shape
    width -= 8
    height -= 8
    dct_pixelwise = np.zeros((width, height, 64))

    T = dct(np.identity(8), norm='ortho')
    for i in range(width):
        for j in range(height):
            dct_pixelwise[i, j, :] = np.reshape(T @ I[i:(i + 8), j:(j + 8)] @ T.T, 64)

    orders = np.zeros((width, height, 64, 4))
    orders[:, :, :, 0] = dct_pixelwise
    orders[:, :, :, 1] = dct_pixelwise**2
    orders[:, :, :, 2] = dct_pixelwise**3
    orders[:, :, :, 3] = dct_pixelwise**4

    presum = np.zeros((width, height, 64, 4))
    for i in range(width):
        for j in range(height):
            if i > 0:
                presum[i, j, :, :] = presum[i, j, :, :] + presum[i - 1, j, :, :]
            if j > 0:
                presum[i, j, :, :] = presum[i, j, :, :] + presum[i, j - 1, :, :]
            if i > 0 and j > 0:
                presum[i, j, :, :] = presum[i, j, :, :] - presum[i - 1, j - 1, :, :]
            presum[i, j, :, :] = presum[i, j, :, :] + orders[i, j, :, :]

    window_size = 32
    pic = np.zeros((width - window_size + 1, height - window_size + 1))
    for i in range(width - window_size + 1):
        for j in range(height - window_size + 1):
            mu = presum[i + 31, j + 31, :, :]
            if i > 0:
                mu = mu - presum[i - 1, j + 31, :, :]
            if j > 0:
                mu = mu - presum[i + 31, j - 1, :, :]
            if i > 0 and j > 0:
                mu = mu + presum[i - 1, j - 1, :, :]

            mu = mu.reshape([64, 4])
            mu = mu / (window_size * window_size)
            mu = mu[1:64, :]
            sig2 = mu[:, 1] - mu[:, 0]**2
            kur = (mu[:, 3]**2 - 4 * mu[:, 2] * mu[:, 0] + 6 * mu[:, 1] * (mu[:, 0]**2) - 3 * (mu[:, 0]**4)) / (
                    mu[:, 1]**2 - 2 * mu[:, 1] * (mu[:, 0]**2) + (mu[:, 0]**4)) - 3

            sqrt_kur = np.sqrt(np.maximum(kur, 0))
            inv_sig2 = 1 / sig2

            c_kur = (np.mean(sqrt_kur) * np.mean(inv_sig2**2) - np.mean(sqrt_kur * inv_sig2) * np.mean(inv_sig2)) / (
                    np.mean(inv_sig2**2) - np.mean(inv_sig2)**2)
            noise_var = (1 - np.mean(sqrt_kur) / c_kur) / np.mean(inv_sig2)
            pic[i, j] = np.sqrt(np.maximum(noise_var, 0))

    return pic

# Example usage
image_path = "/content/nikond70_kodakdcs330_sub_05.tif"
image = cv2.imread(image_path)
result = local_noise_var(image)
plt.imshow(result)  # Choose a colormap that suits your visualization
plt.colorbar()  # Add a colorbar to show the scale
plt.title('Local Noise Level')
plt.show()



In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from skimage.color import label2rgb
import matplotlib.pyplot as plt


# Load the image
I = cv2.imread("/content/nikond70_kodakdcs330_sub_05.tif")

# Calculate local noise variance
aacc = local_noise_var(I)

# Convert to single precision for K-means
aacc = aacc.astype(np.float32)

# Calculate mean of local noise variance
m = np.mean(aacc)

# Reshape the image for clustering
reshaped_image = aacc.reshape((-1, 1))

# Apply k-means clustering
kmeans = KMeans(n_clusters=2)
kmeans.fit(reshaped_image)

# Get cluster labels
labels = kmeans.labels_

# Reshape labels back to the original image shape
label_image = labels.reshape(aacc.shape)

# Overlay labels on the original image
segmented_image = label2rgb(label_image, image=aacc)

# Display the results
plt.imshow(segmented_image)
plt.show()
